In [1]:
from pyspark.sql import SparkSession
import psycopg2

In [2]:
packages = {
   'org.postgresql:postgresql:42.6.0'
}

In [3]:
spark = (
    SparkSession.builder
    .appName('postgres')
    .config('spark.jars.packages', ','.join(packages))
    .getOrCreate()    
)

:: loading settings :: url = jar:file:/home/hduser/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hduser/.ivy2/cache
The jars for the packages stored in: /home/hduser/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-74cd57d3-c233-4703-881b-e091aa2ba7ce;1.0
	confs: [default]
	found org.postgresql#postgresql;42.6.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
downloading https://repo1.maven.org/maven2/org/postgresql/postgresql/42.6.0/postgresql-42.6.0.jar ...
	[SUCCESSFUL ] org.postgresql#postgresql;42.6.0!postgresql.jar (280ms)
downloading https://repo1.maven.org/maven2/org/checkerframework/checker-qual/3.31.0/checker-qual-3.31.0.jar ...
	[SUCCESSFUL ] org.checkerframework#checker-qual;3.31.0!checker-qual.jar (95ms)
:: resolution report :: resolve 513ms :: artifacts dl 379ms
	:: modules in use:
	org.checkerframework#checker-qual;3.31.0 from central in [default]
	org.postgresql#postgresql;42.6.0 from central in [default]
	-------------------------------

23/04/06 14:32:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [11]:
conn = psycopg2.connect(
    host='postgres',
    database='postgres',
    user='postgres',
    password='postgres'
)
cur = conn.cursor()

In [12]:
cur.execute("DROP TABLE IF EXISTS mytable;")
cur.execute("""
    CREATE TABLE IF NOT EXISTS mytable (
        id SERIAL PRIMARY KEY,
        name TEXT NOT NULL,
        age INTEGER NOT NULL
    );
""")

## insert rows into the table
cur.execute("INSERT INTO mytable (name, age) VALUES (%s, %s)", ('John', 25))
cur.execute("INSERT INTO mytable (name, age) VALUES (%s, %s)", ('Mary', 30))

# Commit the changes to the database
conn.commit()

# Close the cursor and database connection
cur.close()
conn.close()

In [13]:
options = {
    'format': 'jdbc',
    'url': 'jdbc:postgresql://postgres:5432/postgres',
    'driver': 'org.postgresql.Driver',
    'user': 'postgres',
    'password': 'postgres',
    'dbtable': 'mytable'
}

In [14]:
df = spark.read.load(**options)

In [15]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



In [16]:
df.show()

+---+----+---+
| id|name|age|
+---+----+---+
|  1|John| 25|
|  2|Mary| 30|
+---+----+---+

